In [111]:
import networkx as nx
import matplotlib as plt
%matplotlib inline 
import math
import random
import numpy as np
from sklearn import preprocessing

In [4]:
#graph = nx.powerlaw_cluster_graph(10000, 2, 0.25, seed=10)

In [5]:
print(graph.edges())

[(0, 2), (0, 1935), (0, 8131), (0, 6), (0, 3735), (0, 6284), (0, 271), (0, 657), (0, 8083), (0, 4500), (0, 149), (0, 23), (0, 8126), (0, 4484), (0, 666), (0, 3631), (0, 4380), (0, 8093), (0, 1567), (0, 5664), (0, 4002), (0, 4132), (0, 5382), (0, 8870), (0, 1787), (0, 5928), (0, 298), (0, 8306), (0, 942), (0, 1327), (0, 8794), (0, 180), (0, 693), (0, 9528), (0, 2335), (0, 4247), (0, 6973), (0, 5438), (0, 2751), (0, 1600), (0, 5671), (0, 450), (0, 4675), (0, 3399), (0, 5320), (0, 3017), (0, 5450), (0, 6476), (0, 5794), (0, 207), (0, 336), (0, 7227), (0, 850), (0, 3411), (0, 6996), (0, 343), (0, 3033), (0, 2010), (0, 1371), (0, 7628), (0, 2910), (0, 610), (0, 1275), (0, 2533), (0, 8294), (0, 4967), (0, 9577), (0, 106), (0, 4007), (0, 2541), (0, 240), (0, 881), (0, 1010), (0, 5491), (0, 5621), (0, 1526), (0, 9720), (0, 1658), (0, 1019), (0, 3242), (0, 8203), (0, 1023), (1, 1537), (1, 2), (1, 3), (1, 4), (1, 5), (1, 7), (1, 9), (1, 10), (1, 11), (1, 525), (1, 15), (1, 16), (1, 17), (1, 18),

In [36]:
numIterations = 10
size = nx.number_of_nodes(graph)

In [27]:
l=[1,2,3]
print(math.sqrt(sum(l)))

2.44948974278


In [37]:
hubScore = [0.0]*size
authority=[1.0]*size


In [61]:
def _random_subset(seq,m):
    """ Return m unique elements from seq.

    This differs from random.sample which can return repeated
    elements if seq holds repeated elements.
    """
    targets=set()
    while len(targets)<m:
        x=random.choice(seq)
        targets.add(x)
    return targets


In [62]:
def powerlaw_cluster_graph(n, m, p, seed=None):
    """Holme and Kim algorithm for growing graphs with powerlaw
    degree distribution and approximate average clustering.

    Parameters
    ----------
    n : int
        the number of nodes
    m : int
        the number of random edges to add for each new node
    p : float,
        Probability of adding a triangle after adding a random edge
    seed : int, optional
        Seed for random number generator (default=None).

    Notes
    -----
    The average clustering has a hard time getting above
    a certain cutoff that depends on m.  This cutoff is often quite low.
    Note that the transitivity (fraction of triangles to possible
    triangles) seems to go down with network size.

    It is essentially the Barabási-Albert (B-A) growth model with an
    extra step that each random edge is followed by a chance of
    making an edge to one of its neighbors too (and thus a triangle).

    This algorithm improves on B-A in the sense that it enables a
    higher average clustering to be attained if desired.

    It seems possible to have a disconnected graph with this algorithm
    since the initial m nodes may not be all linked to a new node
    on the first iteration like the B-A model.

    References
    ----------
    .. [1] P. Holme and B. J. Kim,
       "Growing scale-free networks with tunable clustering",
       Phys. Rev. E, 65, 026107, 2002.
    """

    if m < 1 or n < m:
        raise nx.NetworkXError(\
              "NetworkXError must have m>1 and m<n, m=%d,n=%d"%(m,n))

    if p > 1 or p < 0:
        raise nx.NetworkXError(\
              "NetworkXError p must be in [0,1], p=%f"%(p))
    if seed is not None:
        random.seed(seed)

    G=nx.empty_graph(m,create_using=nx.DiGraph()) # add m initial nodes (m0 in barabasi-speak)
    G.name="Powerlaw-Cluster Graph"
    repeated_nodes=G.nodes()  # list of existing nodes to sample from
                           # with nodes repeated once for each adjacent edge
    source=m               # next node is m
    while source<n:        # Now add the other n-1 nodes
        possible_targets = _random_subset(repeated_nodes,m)
        # do one preferential attachment for new node
        target=possible_targets.pop()
        G.add_edge(source,target)
        repeated_nodes.append(target) # add one node to list for each new link
        count=1
        while count<m:  # add m-1 more new links
            if random.random()<p: # clustering step: add triangle
                neighborhood=[nbr for nbr in G.neighbors(target) \
                               if not G.has_edge(source,nbr) \
                               and not nbr==source]
                if neighborhood: # if there is a neighbor without a link
                    nbr=random.choice(neighborhood)
                    G.add_edge(source,nbr) # add triangle
                    repeated_nodes.append(nbr)
                    count=count+1
                    continue # go to top of while loop
            # else do preferential attachment step if above fails
            target=possible_targets.pop()
            G.add_edge(source,target)
            repeated_nodes.append(target)
            count=count+1

        repeated_nodes.extend([source]*m)  # add source node to list m times
        source += 1
    return G

In [63]:
graph = powerlaw_cluster_graph(10000, 2, 0.25, seed=10)

In [28]:
def normalize(l):
    RSS=math.sqrt(sum([i*i for i in l]))
    return [i/RSS for i in l]

In [41]:
hits, authority = nx.hits(graph, max_iter=500)

    

In [64]:
print type(graph)
G=nx.empty_graph(10,create_using=nx.DiGraph())
print type(G)

<class 'networkx.classes.digraph.DiGraph'>
<class 'networkx.classes.digraph.DiGraph'>


In [65]:
inDegree = graph.in_degree()
outDegree = graph.out_degree()

In [78]:
revGraph=graph.reverse(copy=True)
adjListDict={}
for node in graph.nodes():
    adjListDict[node]=revGraph.neighbors(node)

In [90]:
featureVector = []
for i in range(0,size):
    featureVector.append([inDegree[i], outDegree[i], hits[i], authority[i]])

print type(featureVector)




<type 'list'>


In [91]:
featureVectorNP = np.asarray(featureVector)

In [112]:
featureVectorScaled = preprocessing.scale(featureVectorNP)

In [121]:
featureVectorScaled[10]

array([  1.81769823e+01,   1.41435500e-02,   1.01420220e+01,
         1.01420220e+01])

In [88]:
print featureVector[9]
x=[1,2,5,0.5]
y=[1,1,1,1]

(294, 2, 0.0063385216943580265, 0.00633852169212081)


TypeError: can't multiply sequence by non-int of type 'list'

In [115]:
def calcSync(node, followers, featureVector):
    if len(followers)==0:
        return 0.0
    numeratorSum = 0.0
    for i in range(0,len(followers)):
        for j in range(i+1, len(followers)):
            numeratorSum += np.dot(featureVector[i],featureVector[j])
    return numeratorSum/(len(followers)*len(followers))

In [116]:
def calcNormalcy(node, followers, featureVector, size):
    numeratorSum = 0.0
    if len(followers)==0:
        return 0.0
    for i in range(0,len(followers)):
        for j in range(0, size):
            if i!=j:
                numeratorSum += np.dot(featureVector[i],featureVector[j])
    return numeratorSum/(len(followers)*size)

In [119]:
calcNormalcy(11, adjListDict[11], featureVectorScaled, size)

-0.33211523224876238

In [120]:
calcSync(11, adjListDict[11], featureVectorScaled)

354.40228279957188

In [107]:
sync=[]
normalcy=[]
for i in range(0, size):
    sync.append(calcSync(i, adjListDict[i], featureVectorNP))


In [108]:
print sync

[767.74948035268983, 87.901594574984614, 350.87100623297175, 486.21074307727793, 791.53694006371211, 2795.2261193553236, 2882.1875894818663, 5482.3258099965924, 4935.448169553526, 361.02732698497584, 552.02360777535341, 19410.468770784439, 4006.7947896348064, 1623.4819396868484, 2273.3022514398963, 1332.5427482921837, 855.52376309004944, 4791.5555615864114, 11082.901247074482, 1486.6093272669298, 7714.2854538222246, 12028.765638159104, 12028.765638159104, 9787.8300114208687, 2500.4320630132747, 5196.0878975321639, 6916.5488083457858, 12028.765638159104, 11082.901247074482, 4791.5555615864114, 898.44305452389017, 4596.3788146137631, 2795.2261193553236, 1911.4147144032943, 1746.720719987414, 3693.006918089693, 2302.209719793459, 3182.3268013900283, 4935.448169553526, 6542.8032627803987, 5059.4389416891081, 11082.901247074482, 2447.2300487700222, 5251.6888958366881, 14446.337295091918, 4293.8581303148603, 12501.10223586248, 9787.8300114208687, 2795.2261193553236, 2564.3596941975597, 24276

In [109]:
for i in range(0, size):
    normalcy.append(calcNormalcy(i, adjListDict[i], featureVectorNP, size))

In [110]:
print normalcy

[82.631189823284089, 30.184116771196084, 56.902761254950882, 66.399801868184284, 83.848115098829737, 156.3171585755629, 158.6937800286191, 221.59524195233234, 208.4087662379676, 57.673047406537442, 70.536130818672888, 453.06938861493745, 187.26157189524432, 119.27059911048703, 140.95492958154426, 108.11312938117408, 87.051419278283404, 205.2245866236193, 317.22450632122968, 114.09332261251366, 263.60644413879083, 333.03491954665657, 333.03491954665657, 297.05389572579196, 147.95354443943103, 214.68289742593694, 249.14104461515569, 333.03491954665657, 317.22450632122968, 205.2245866236193, 89.178989720252432, 200.66755313074515, 156.3171585755629, 129.41795237556147, 123.73041928780516, 179.65470710701493, 141.87873941795561, 166.61784943267938, 208.4087662379676, 242.16960445854696, 211.62203085175599, 317.22450632122968, 146.27058724651133, 216.73722722240169, 370.22072399535278, 193.87875297942927, 341.31080081768232, 297.05389572579196, 156.3171585755629, 149.81551589422091, 508.950